<a href="https://colab.research.google.com/github/LEFT-BEE/small_project/blob/main/%EC%88%98%EC%B9%98%ED%95%B4%EC%84%9Dproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adaboost 
1. 각 weak 모델에서 학습할 데이터 선택
2. 모든 데이터의 가중치 초기화
3. 1회 학습 후 예측 오류(error)계산, 가중치(a)계산, 가중치(D)갱신
4. 반복 회수별로 가중치 갱신
5. 모든 모델이 위의 단계를 수행할때 까지 반복

가중치(D) : 모든 train 데이터에 적용(초기값 동일)

오류(e) : 오류데이터 / 전체 학습데이터 , 각 모델의 오류

모델별 가중치(a) : ln((1-e) / e) / 2 , 오류를 기반으로 계산

예측이 맞을 경우 -> $D_i^{t+1} = \frac {D_i^{t} \times e^{-a} }{Sum(D)} $

예측이 틀린 경우 -> $D_i^{t+1} = \frac {D_i^{t} \times e^{a} }{Sum(D)} $

강분류기 = 이전단계의 약분류기 + $W_t \times H_t$
$H(x) = sign(\alpha_1h_1(x) + \alpha_2h_2(x) ... \alpha_Nh_N(x)$

즉 약한 분류기가 여러번 약점보완을 통해 강력한 분류기가된다.

### 20181512 한승진






In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_hastie_10_2
import matplotlib.pyplot as plt


def get_error_rate(pred, Y):
    return sum(pred != Y) / float(len(Y))


def print_error_rate(err):
    print ('Error rate: Training: %.4f - Test: %.4f' % err)

error_rate = 예측실패한 데이터의수 / 전체데이터의수

In [ ]:
def generic_clf(Y_train, X_train, Y_test, X_test, clf):
    clf.fit(X_train,Y_train)
    pred_train = clf.predict(X_train)
    pred_test = clf.predict(X_test)
    return get_error_rate(pred_train, Y_train), get_error_rate(pred_test, Y_test)

In [ ]:
def adaboost_clf(Y_train, X_train, Y_test, X_test, M, clf):
    n_train, n_test = len(X_train), len(X_test)

    # 가중치 초기화
    w = np.ones(n_train) / n_train
    pred_train, pred_test = [np.zeros(n_train), np.zeros(n_test)]
    
    for i in range(M):
        clf.fit(X_train, Y_train, sample_weight = w)
        pred_train_i = clf.predict(X_train)
        pred_test_i = clf.predict(X_test)

        # Indicator function
        miss = [int(x) for x in (pred_train_i != Y_train)]
        miss2 = [x if x==1 else -1 for x in miss]

        # Error
        err_m = np.dot(w,miss) / sum(w)
        # Alpha
        alpha_m = 0.5 * np.log( (1 - err_m) / float(err_m))
        # New weights
        w = np.multiply(w, np.exp([float(x) * alpha_m for x in miss2]))

        # 이부분은 copy and paste하였습니다 zip함수의 유용성을 꺠닫게 되었네요
        pred_train = [sum(x) for x in zip(pred_train, 
                                          [x * alpha_m for x in pred_train_i])]
        pred_test = [sum(x) for x in zip(pred_test, 
                                         [x * alpha_m for x in pred_test_i])]
    #예측값들 sign함수로 이진분류해주었다.
    pred_train, pred_test = np.sign(pred_train), np.sign(pred_test)

    # Return error rate in train and test set
    return get_error_rate(pred_train, Y_train),get_error_rate(pred_test, Y_test)

Adaboost Algorithm (is not copy and paste)

1. 가중치를 초기화 해준다 : 
$w_i = 1/N , i = 1,2,...,N$

2. $G_m(x)$를 학습시킨다 , 이떄 $G_m(x)$는 sklearn의 Decision_tree classifer를 가져와 사용하였다.

3. 학습한 결과의 Total  error를 찾고 이는 Amount of Say를 결정한다 

  $err_m =\frac{\sum_{i=1}^{t+1} w_i I(y_i \neq G_m(x_i)))}{\sum_{i=1}^N w_i}$

  $\alpha = log((1-err_m) / err_m) $

4. 예측이 틀린 경우의 가중치를 업데이트 해준다
  $w_i \leftarrow w_i \cdot exp [ \alpha_m \cdot I(y_i \neq G_m(x_i))] $

  cf)가중치의 경우 논문에서는 두가지 경우로 나누어 가중치를 업데이트하는 방식이 달랐는데 어떤 케이스는 틀린부분의 케이스 만을 업데이트 하는 방식도 있다.. 

5. 학습한 분류 결과의 합을 sign함수를 통해 이진분류값으로 바꾼다 

  $Output G(x) = sign[\sum_{m=1}^M \alpha_m G_m(x)]$

### 20181512 한승진




In [ ]:
def plot_error_rate(er_train, er_test , mystring):
    df_error = pd.DataFrame([er_train, er_test]).T
    df_error.columns = ['Training', 'Test']
    plot1 = df_error.plot(linewidth = 3, figsize = (8,6),
            color = ['lightblue', 'darkblue'], grid = True)
    plot1.set_xlabel('Number of iterations', fontsize = 12)
    plot1.set_ylabel('Error rate', fontsize = 12)
    plot1.set_title(mystring, fontsize = 16)
    plt.axhline(y=er_test[0], linewidth=1, color = 'red', ls = 'dashed')

In [ ]:
""" =================================TEST CASE 1 : RANDOM SCATTER DATA ================================="""
if __name__ == '__main__':
    
    #Hastie et al.에서 사용 된 2 진 분류를위한 데이터를 생성합니다.10 개의 피쳐는
    # 표준 독립적 인 가우시안이며 타겟 y은 다음에 의해 정의됩니다.
    #12000천개의 임의 데이터
    x, y = make_hastie_10_2()
    df = pd.DataFrame(x)
    #dataframe에 label축 추가
    df['Y'] = y

    # 편리한 train_test_split 모듈을 사용해 train과 test를 나눈다.
    train, test = train_test_split(df, test_size = 0.2)
    X_train, Y_train = train.iloc[:,:-1], train.iloc[:,-1]
    X_test, Y_test = test.iloc[:,:-1], test.iloc[:,-1]
    
    # decisiontree는 모듈을 이용하였습니다.
    #adaboost tree 모델생성
    clf_tree = DecisionTreeClassifier(max_depth = 2, random_state = 1)
    er_tree = generic_clf(Y_train, X_train, Y_test, X_test, clf_tree)
    
    er_train, er_test = [er_tree[0]], [er_tree[1]]
  
    for i in range(10 , 100):    
        print(i-9,"번째 epoch")
        er_i = adaboost_clf(Y_train, X_train, Y_test, X_test, i , clf_tree)
        print_error_rate(er_i)
        er_train.append(er_i[0])
        er_test.append(er_i[1])
    
    # Compare error rate vs number of iterations
    plot_error_rate(er_train, er_test , "Error rate , TEST CASE 1")

In [ ]:
""" ==========================TEST CASE 2 : SKLEARN - CANCER DATA================================="""
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
#data는 유방암 분류 데이터 셋
if __name__ == '__main__':
    
    x = data.data
    y = data.target

    df = pd.DataFrame(x)
    #dataframe에 label축 추가
    df['Y'] = y

    # 편리한 train_test_split 모듈을 사용해 train과 test를 나눈다.
    train, test = train_test_split(df, test_size = 0.2)
    X_train, Y_train = train.iloc[:,:-1], train.iloc[:,-1]
    X_test, Y_test = test.iloc[:,:-1], test.iloc[:,-1]

    clf_tree = DecisionTreeClassifier(max_depth = 2, random_state = 1)
    er_tree = generic_clf(Y_train, X_train, Y_test, X_test, clf_tree)


    for i in range(100 , 105):    
        print(i-99,"번째 epoch")
        er_i = adaboost_clf(Y_train, X_train, Y_test, X_test, i, clf_tree)
        print_error_rate(er_i)
        er_train.append(er_i[0])
        er_test.append(er_i[1])
    
    # Compare error rate vs number of iterations
    plot_error_rate(er_train, er_test , "Error rate , TEST CASE 2")
  